In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
def analyze_dataset(df, filename):
    """Analyze dataset for preprocessing requirements"""
    results = {'filename': filename}
    
    # Column spacing issues
    spacing_issues = sum(1 for col in df.columns if col.startswith(' ') or col.endswith(' '))
    results['spacing_issues'] = spacing_issues
    
    # Metadata columns
    metadata_cols = []
    if ' Destination Port' in df.columns:
        metadata_cols.append(' Destination Port')
    results['metadata_cols'] = len(metadata_cols)
    
    # Duplicate columns
    duplicate_cols = []
    if ' Fwd Header Length.1' in df.columns:
        duplicate_cols.append(' Fwd Header Length.1')
    results['duplicate_cols'] = len(duplicate_cols)
    
    # Missing/infinite values
    df_temp = df.replace([np.inf, -np.inf], np.nan)
    rows_with_issues = df_temp.isnull().any(axis=1).sum()
    results['rows_with_issues'] = rows_with_issues
    
    # Duplicates
    feature_cols = df.columns.difference([' Label'])
    duplicates = df.duplicated(subset=feature_cols).sum()
    results['duplicates'] = duplicates
    
    # Label inconsistencies
    label_issues = 0
    if ' Label' in df.columns and 'BENIGN' in df[' Label'].unique():
        label_issues = 1
    results['label_issues'] = label_issues
    
    # Expected final shape
    expected_rows = len(df) - rows_with_issues - duplicates
    expected_cols = len(df.columns) - len(metadata_cols) - len(duplicate_cols)
    results['original_shape'] = df.shape
    results['expected_shape'] = (expected_rows, expected_cols)
    
    return results

def explore_dataset(df, name="Dataset"):
    """Quick dataset exploration"""
    print(f"\n{name} Analysis")
    print("="*50)
    
    print(f"Columns ({len(df.columns)}):")
    print(df.columns.tolist())
    print("="*80)
    
    print("\nDataset Info:")
    print(df.info())
    
    if 'Label' in df.columns or ' Label' in df.columns:
        label_col = 'Label' if 'Label' in df.columns else ' Label'
        print(f"\nClass Distribution:")
        class_counts = df[label_col].value_counts()
        total = len(df)
        for label, count in class_counts.items():
            pct = (count / total) * 100
            print(f"{label}: {count:,} ({pct:.1f}%)")
        
        max_count = class_counts.max()
        print("\nClass Imbalance Ratios:")
        for label, count in class_counts.items():
            ratio = max_count / count
            print(f"{label}: 1:{ratio:.2f}")
    
    missing = df.isnull().sum()
    missing_cols = missing[missing > 0]
    if len(missing_cols) > 0:
        print("\nMissing Values:")
        print(missing_cols)
    else:
        print("\nNo missing values found.")

In [3]:
# Get all CSV files
data_path = '/home/sagemaker-user/cybersecurity-tensor-ad/data/raw/cicids2017/'
csv_files = glob.glob(os.path.join(data_path, '*.csv'))

print(f"Found {len(csv_files)} CSV files")
print("\nAnalyzing each file...")

all_results = []
for file_path in csv_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")
    
    try:
        df = pd.read_csv(file_path)
        explore_dataset(df, filename)
        
        results = analyze_dataset(df, filename)
        all_results.append(results)
        
        print(f"  Shape: {results['original_shape']}")
        print(f"  Issues: {results['spacing_issues']} spacing, {results['rows_with_issues']} NaN/inf, {results['duplicates']} duplicates")
        
        # Clear memory
        del df
        gc.collect()
        
    except Exception as e:
        print(f"  Error: {e}")
        gc.collect()

Found 8 CSV files

Analyzing each file...

Processing: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv

Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv Analysis
Columns (79):
[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', 

In [4]:
# Summary across all files
if all_results:
    summary_df = pd.DataFrame(all_results)
    
    print("=" * 80)
    print("SUMMARY ACROSS ALL FILES")
    print("=" * 80)
    
    print(f"\nTotal files analyzed: {len(summary_df)}")
    print(f"Files with spacing issues: {(summary_df['spacing_issues'] > 0).sum()}")
    print(f"Files with metadata columns: {(summary_df['metadata_cols'] > 0).sum()}")
    print(f"Files with duplicate columns: {(summary_df['duplicate_cols'] > 0).sum()}")
    print(f"Files with NaN/inf issues: {(summary_df['rows_with_issues'] > 0).sum()}")
    print(f"Files with duplicates: {(summary_df['duplicates'] > 0).sum()}")
    print(f"Files with label issues: {(summary_df['label_issues'] > 0).sum()}")
    
    print("\nDetailed results:")
    display(summary_df[['filename', 'original_shape', 'spacing_issues', 'metadata_cols', 
                       'duplicate_cols', 'rows_with_issues', 'duplicates', 'expected_shape']])

SUMMARY ACROSS ALL FILES

Total files analyzed: 8
Files with spacing issues: 8
Files with metadata columns: 8
Files with duplicate columns: 8
Files with NaN/inf issues: 8
Files with duplicates: 8
Files with label issues: 8

Detailed results:


,filename,original_shape,spacing_issues,metadata_cols,duplicate_cols,rows_with_issues,duplicates,expected_shape
0,Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv,"(225745, 79)",65,1,1,34,2633,"(223078, 77)"
1,Friday-WorkingHours-Afternoon-PortScan.pcap_IS...,"(286467, 79)",65,1,1,371,72353,"(213743, 77)"
2,Friday-WorkingHours-Morning.pcap_ISCX.csv,"(191033, 79)",65,1,1,122,6888,"(184023, 77)"
3,Monday-WorkingHours.pcap_ISCX.csv,"(529918, 79)",65,1,1,437,26935,"(502546, 77)"
4,Thursday-WorkingHours-Morning-WebAttacks.pcap_...,"(170366, 79)",65,1,1,135,6066,"(164165, 77)"
5,Thursday-WorkingHours-Afternoon-Infilteration....,"(288602, 79)",65,1,1,207,35630,"(252765, 77)"
6,Tuesday-WorkingHours.pcap_ISCX.csv,"(445909, 79)",65,1,1,264,24065,"(421580, 77)"
7,Wednesday-workingHours.pcap_ISCX.csv,"(692703, 79)",65,1,1,1297,81957,"(609449, 77)"
